## Using multicore TPU to accelerate Neural Network Trianing





**Step 1.**
Install the pytorch XLA libreries to communicate with the TPU





In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

     |████████████████████████████████| 144.6MB 27kB/s 
     |████████████████████████████████| 61kB 3.5MB/s 
ERROR: earthengine-api 0.1.260 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  Found existing installation: google-api-python-client 1.12.8
    Uninstalling google-api-python-client-1.12.8:
      Successfully uninstalled google-api-python-client-1.12.8


In [3]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split
from torch.utils.data.distributed import DistributedSampler
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA

# import torch xla APIs

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

In [4]:
import pickle

In [5]:
#train_size=int(0.4*(unprocessed_data.shape[0]))
# val_size=len(unprocessed_data) - train_size
# print(train_size,val_size)
# train_ds,_ = random_split(unprocessed_data,[train_size,val_size])

**Step 2.** Get the dataset

---



In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
#unscaled = np.load('/content/drive/MyDrive/Autoencoding/Unscaled_data_sorted.npy')
#!cd /content/drive/MyDrive/

In [8]:
ls -lsrt

total 8
4 drwxr-xr-x 1 root root 4096 May  6 13:44 sample_data/
4 drwx------ 6 root root 4096 May 10 06:03 drive/


In [9]:
#train_dataset = unscaled[unscaled[:,0] < 2.0]
#train_dataset = pickle.load(open('all_clw_1780_4047_reshaped.pkl','rb'))

In [10]:
#train_dataset.shape

In [11]:
# pca_whiten = PCA(whiten=True)

# train_dataset = pca_whiten.fit_transform(train_dataset)

In [12]:
#train_dataset = torch.from_numpy(train_dataset)

In [13]:
#train_dataset =  torch.load('/content/drive/MyDrive/Autoencoding/raw_cv_whiten.pt')
#torch.save(train_dataset,'xyz_traj_whitten.pt')
train_dataset = torch.load('/content/drive/MyDrive/1780_to_4047_files/xyz_traj_whitten.pt')

In [14]:
# Define Parameters
FLAGS = {}
#FLAGS['data_dir'] = "/tmp/cifar"
FLAGS['batch_size'] = 2056
FLAGS['num_workers'] = 4
FLAGS['max_learning_rate'] = 0.001
#FLAGS['grad_clip']  = 0.1
FLAGS['weight_decay'] = 1e-4
FLAGS['opt_func']  = torch.optim.Adam
#FLAGS['momentum'] = 0.9
FLAGS['num_epochs'] = 5
FLAGS['num_cores'] = 8 
FLAGS['log_steps'] = 20
FLAGS['metrics_debug'] = False

** warp the dataloader for parallelization**

In [15]:
#SERIAL_EXEC = xmp.MpSerialExecutor()

In [16]:
#train_dl = DataLoader(train_ds,batch_size,shuffle=True)

In [17]:
# def make_4_dim(data):
#     data=torch.unsqueeze(data,1)
#     data = torch.unsqueeze(data,3)
#     return data

# def make_2_dim(data):
#     data=torch.squeeze(data,1)
#     data = torch.squeeze(data,2)
#     return data

In [18]:
def lin_block(in_dim,out_dim,bias=True):
    layers = [nn.Linear(in_dim,out_dim,bias= True),
              #nn.BatchNorm2d(out_dim),
              nn.LeakyReLU(negative_slope=0.2,inplace=True)]
    return nn.Sequential(*layers)

# def conv_block2(in_channels,out_channels,kernel_size,stride,padding,pool=False):
#     layers = [nn.Linear(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,
#                         stride=stride,padding=padding),
#               nn.BatchNorm2d(out_channels),
#               nn.LeakyReLU(negative_slope=0.2,inplace=True)]
#     return nn.Sequential(*layers)
    


256


In [75]:
#hidden_layers = [64,128,256]
class Resnet9(nn.Module):
      def __init__(self, in_dim,hidden_layers):
        super().__init__()
        
        # Encode1
        
        self.lin1 = lin_block(in_dim,hidden_layers[0])         
        self.lin2 = lin_block(hidden_layers[0],hidden_layers[1])
        self.lin3 = lin_block(hidden_layers[1],hidden_layers[2]) 
        
        self.res1 = nn.Sequential(lin_block(hidden_layers[2], hidden_layers[2]),  
                                  lin_block(hidden_layers[2], hidden_layers[2]))
        
        #Encode2
        self.lin4 = lin_block(hidden_layers[2],hidden_layers[1])         
        self.lin5 = lin_block(hidden_layers[1],hidden_layers[0])
        
        self.res2 = nn.Sequential(lin_block(hidden_layers[0], hidden_layers[0]),  
                                  lin_block(hidden_layers[0], hidden_layers[0]))
        self.lin6 = nn.Linear(hidden_layers[0],in_dim)
        
         # Decode1
        
        self.lin7 = lin_block(in_dim,hidden_layers[0])         
        self.lin8 = lin_block(hidden_layers[0],hidden_layers[1])
        self.lin9 = lin_block(hidden_layers[1],hidden_layers[2]) 
        
        self.res3 = nn.Sequential(lin_block(hidden_layers[2], hidden_layers[2]),  
                                  lin_block(hidden_layers[2], hidden_layers[2]))
        
        #Decode2
        self.lin10 = lin_block(hidden_layers[2],hidden_layers[1])         
        self.lin11 = lin_block(hidden_layers[1],hidden_layers[0])
        
        self.res4 = nn.Sequential(lin_block(hidden_layers[0], hidden_layers[0]),  
                                  lin_block(hidden_layers[0], hidden_layers[0]))
        self.lin12 = nn.Linear(hidden_layers[0],in_dim)
        
      def encode(self,in_data):
          out = self.lin1(in_data.float())
          out = self.lin2(out)
          out = self.lin3(out)
          out = self.res1(out)+out
          out = self.lin4(out)
          out = self.lin5(out)
          out = self.res2(out)+out
          out = self.lin6(out)
          return out
           
      def decode(self,lat_data):
          out = self.lin7(lat_data.float())
          out = self.lin8(out)
          out = self.lin9(out)
          out = self.res3(out)+out
          out = self.lin10(out)
          out = self.lin11(out)
          out = self.res4(out)+out
          out = self.lin12(out)
          return out
      
        

In [100]:
hidden_layers = [256,128,64,4]
in_dim = 2
print(hidden_layers[0])

256


In [101]:
WRAPPED_MODEL = xmp.MpModelWrapper(Resnet9(in_dim,hidden_layers))

In [102]:
# # def get_default_device():
# #     """Pick GPU if available, else CPU"""
# #     if torch.cuda.is_available():
# #         return torch.device('cuda')
# #     else:
# #         return torch.device('cpu')
    
# def to_device(data, device):
#     """Move tensor(s) to chosen device"""
#     if isinstance(data, (list,tuple)):
#         return [to_device(x, device) for x in data]
#     return data.to(device, non_blocking=True)

# class DeviceDataLoader():
#     """Wrap a dataloader to move data to a device"""
#     def __init__(self, dl, device):
#         self.dl = dl
#         self.device = device
        
#     def __iter__(self):
#         """Yield a batch of data after moving it to device"""
#         for b in self.dl: 
#             yield to_device(b, self.device)

#     def __len__(self):
#         """Number of batches"""
#         return len(self.dl)

In [103]:
lag = 8
train_data = train_dataset[:-lag]
shifted_train_data = train_dataset[lag:]
transformed_train_data = torch.hstack((train_data,shifted_train_data))

In [104]:
def training_step(data,encoder):
    training_dat = data[:,0:2]
    #training_dat = make_4_dim(training_dat)
    shifted_dat = data[:,2:4]
    out = encoder.encode(training_dat)
    out = encoder.decode(out)
    #out = make_2_dim(out)
    criterion = nn.MSELoss()
    loss = criterion(out.float(),shifted_dat.float())
    return loss

In [105]:
# def training_step(data,encoder):
#     training_dat = data
#     training_dat = make_4_dim(training_dat)
#     out = encoder.encode(training_dat)
#     out = encoder.decode(out)
#     out = make_2_dim(out)
#     criterion = nn.MSELoss()
#     loss = criterion(out.float(),data.float())
#     return loss

# def evaluate(model,val_loader):
#     for data in val_loader:
#         val_dat = data
#         val_dat = make_4_dim(val_dat)
#         out = model.encode(val_dat)
#         out = model.decode(out)
#         out = make_2_dim(out)
#         criterion = nn.MSELoss()
#         loss = criterion(out.float(),data.float())
#         return {'val_loss':loss}

In [106]:
from tqdm.notebook import tqdm

In [107]:
# def prepare_data(unprocessed_data):

#       train_ds = unprocessed_data[:]
#       train_ds = train_ds -torch.mean(train_ds,0) # Mean free

#       pca_whiten = PCA(whiten=True)                     # Whiten data
#       train_ds = pca_whiten.fit_transform(train_ds)

#       train_ds = torch.tensor(train_ds)                  # as tensor
#       return train_ds

In [108]:
#@torch.no_grad()
def fit_one_cycle(FLAGS):
    
    torch.manual_seed(1234)
    history = []

    

    #train_dataset = SERIAL_EXEC.run(prepare_data(unprocessed_data))
    #train_dataset = prepare_data(unprocessed_data)
    train_sampler = DistributedSampler(transformed_train_data,
                                       num_replicas=xm.xrt_world_size(),
                                       #num_replicas=8,
                                       rank= xm.get_ordinal(),
                                       shuffle=True)
    train_loader  = DataLoader(transformed_train_data,
                               batch_size=FLAGS['batch_size'],
                               sampler= train_sampler,
                               num_workers=FLAGS['num_workers'])
    
    # Scale learning rate to no of torch devices
    max_lr = FLAGS['max_learning_rate']*xm.xrt_world_size()

    # Get loss function, optimizer, and model
    device = xm.xla_device()
    encoder = WRAPPED_MODEL.to(device)
    opt_func = FLAGS['opt_func']
    # set up custom optimizer with weight decay
    optimizer = opt_func(encoder.parameters(),
                         FLAGS['max_learning_rate'],
                         weight_decay=FLAGS['weight_decay'])
    
    # set up one_cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer,
                                                FLAGS['max_learning_rate'],
                                               epochs=FLAGS['num_epochs'],
                                               steps_per_epoch=len(train_loader))
    
    
    # crate the loop for training
    
    for epoch in range(FLAGS['num_epochs']):
        # Training Phase
        encoder.train()
        train_losses = []
        para_loader = pl.ParallelLoader(train_loader,[device]).per_device_loader(device)
        #train_loop_fn(para_loader.per_device_loader(device))
        xm.master_print("Finished training epoch {}".format(epoch))
        
        for batch in tqdm(para_loader):
            
            loss = training_step(batch,encoder)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            #if grad_clip:
            #nn.utils.clip_grad_value_(encoder.parameters(),
            #                             FLAGS['grad_clip'])
            xm.optimizer_step(optimizer)
            optimizer.zero_grad()
            
            sched.step()
        # Validation Phase    
        #result = evaluate(model,val_loader)
        train_loss = torch.stack(train_losses).mean().item()
        print('train_loss{:.4f}'.format(train_loss))
        history.append(train_loss)
        history_ten = torch.tensor(history)
        xm.save(encoder.state_dict(),'encoder_state_dict_resnet.pth')
        xm.save(history_ten,'history_resnet.pth')
    return history_ten

In [109]:
train_sampler = DistributedSampler(transformed_train_data,
                                       num_replicas=xm.xrt_world_size(),
                                       #num_replicas=8,
                                       rank= xm.get_ordinal(),
                                       shuffle=True)
train_loader  = DataLoader(transformed_train_data,
                               batch_size=FLAGS['batch_size'],
                               sampler= train_sampler,
                               num_workers=FLAGS['num_workers'])
for batch in train_loader:
    print(batch.shape)
    break

torch.Size([2056, 612])


## Make the Map Function 

In [110]:
#torch.unsqueeze(batch,dim=0).shape

In [111]:
def map_fn(rank,flags):
    global FLAGS
    FLAGS = flags 
    torch.set_default_tensor_type('torch.FloatTensor')
    #history = []
    history_ten = fit_one_cycle(FLAGS)
    # if rank == 0:
    # # Retrieve tensors that are on TPU core 0 and plot.
    #    xm.save(encoder_state_dict,'encoder_state_dict.pth')

if __name__ == '__main__':
          xmp.spawn(map_fn,args = (FLAGS,),nprocs=FLAGS['num_cores'],
          start_method='fork')

Finished training epoch 0


Exception in device=TPU:6: tensorflow/compiler/xla/xla_client/mesh_service.cc:331 : Failed to retrieve mesh configuration: Connection reset by peer (14)
Exception in device=TPU:3: tensorflow/compiler/xla/xla_client/mesh_service.cc:331 : Failed to retrieve mesh configuration: Connection reset by peer (14)
Exception in device=TPU:7: tensorflow/compiler/xla/xla_client/mesh_service.cc:331 : Failed to retrieve mesh configuration: Connection reset by peer (14)
Exception in device=TPU:5: tensorflow/compiler/xla/xla_client/mesh_service.cc:331 : Failed to retrieve mesh configuration: Connection reset by peer (14)
Exception in device=TPU:2: tensorflow/compiler/xla/xla_client/mesh_service.cc:331 : Failed to retrieve mesh configuration: Connection reset by peer (14)
Exception in device=TPU:4: tensorflow/compiler/xla/xla_client/mesh_service.cc:331 : Failed to retrieve mesh configuration: Connection reset by peer (14)
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (m

ProcessExitedException: ignored

In [ ]:
history = torch.load('history_resnet.pth')

In [ ]:
plt.plot(history)
plt.xlabel = ('Epochs')
plt.ylabel = ('MSE_loss')
plt.savefig('loss.png')

In [ ]:
encoder = Resnet9(1,1);decoder =Resnet9(1,1)

In [ ]:
decoder.load_state_dict(torch.load('encoder_state_dict.pth'))
encoder.load_state_dict(torch.load('encoder_state_dict_1.pth'))

In [ ]:
torch.all(torch.eq(decoder.state_dict(),encoder.state_dict()))

In [ ]:
encoder.encode

In [ ]:
import torch
history = torch.load('history_4.pth')


In [ ]:
history

In [ ]:
  for epoch in range(FLAGS['num_epochs']):
    print (epoch)